# SQL DATA ANALYSIS TASK

# Context
Fenix is a company that is in renewable energy. To this end, it sells solar kits to populations who are in need. Customers have the option of paying for the kit in cash or on credit. If the customer chooses to buy on credit, he can choose between one year (360 days) or two years (720). He is then asked to pay an amount per day (daily_rate), this amount depending on the duration of the loan and the type of kit he has chosen and corresponds to a day of light. So every time he pays that amount, he has a day of light. After 24 hours the system hangs and it has to make another payment to take advantage of the light. He can decide to pay for more than one day of light. The day number is then the whole part of the amount / daily rate ratio. After this number of days, the system still locks. When a customer spends more than 30 days without light, he is considered inactive. Paid vs Expected (PvE) means the ratio between the total amount paid by the client and the amount he should have paid up to date. It allows to appreciate the recovery.

Let's consider the following tables

Loan_current_daitails

loan_id	|customer_id|daily_rate|days_elapsed|Days_til_lock_out|Total_paid
--------|-----------|----------|------------|-----------------|----------
loan_id_1|customer_id_1|daily_rate_1|days_elapsed_1|Days_til_lock_out_1|Total_paid_1
loan_id_2|customer_id_2|daily_rate_2|days_elapsed_2|Days_til_lock_out_2|Total_paid_2
.........|.............|............|..............|...................|............
loan_id_n|customer_id_n	|daily_rate_n|days_elapsed_n|Days_til_lock_out_n|Total_paid_n


Person_democraphic

customer_id	|country|	region|	profession
------------|-------|---------|-----------
customer_id_1|	Country_1	|Region_1	|profession_1
.............|	............|..........	|............
customer_id_j|	Country_j|	Region_j	|profession_j


Sales_details

loan_id	|Product_type	|Date_fullfiled	|loan_duration
--------|----------------|--------------|-------------
loan_id_1	|Product_type_1	|Date_fullfiled_1	|loan_duration_1
...|	...	|...	|...
loan_id_n|	Product_type_n|	Date_fullfiled_n|	loan_duration_n

### Tables description

* **Loan_id:** unique loan identifier

* **Customer_id:** unique customer identifier

* **Days_elapsed:** the number of days the client spends with us

* **Daily_rate:** expected amount from customer per day

* **Days_til_lock_out:** If positive, then corresponds to the number of days of light remaining before the system hangs. If negative, corresponds to the number of days since the system was blocked (the client has been without light since this number of days)

* **Total_paid:** total amount to be paid by the customer

* **Country:** Customer's country

* **Region:** customer department

* **Profession:** client's profession

* **Product_type:** the type of kit purchased by the customer

* **Loan_duration:** the duration of the customer's loan (360 or 720 days)

* **Date_fullfiled:** the date of the sale

### The datasets
The datasets exist in csv formats and can be downloaded and saved on disk

In [42]:

%load_ext sql
%sql sqlite:///testdb.sqlite

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [43]:
%%sql
DROP TABLE IF EXISTS Loan_current_daitails;
DROP TABLE IF EXISTS Person_democraphic;
DROP TABLE IF EXISTS Sales_details;

 * sqlite:///testdb.sqlite
Done.
Done.
Done.


[]

In [44]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("testdb.sqlite")
customers = pd.read_csv('Loan_current_daitails.csv')
orders = pd.read_csv('Person_democraphic.csv')
salesman = pd.read_csv('Sales_details.csv')

customers.to_sql("Loan_current_daitails", conn, index=False, if_exists="replace")
orders.to_sql("Person_democraphic", conn, index=False, if_exists="replace")
salesman.to_sql("Sales_details", conn, index=False, if_exists="replace")

c:\users\gkkar\appdata\local\programs\python\python36\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


### Write a SQL code to determine the total number of loans on the one hand and the total number of customers on the other

In [45]:
%%sql
SELECT * FROM Loan_current_daitails LIMIT 2;

 * sqlite:///testdb.sqlite
Done.


loan_id,customer_id,daily_rate,days_elapsed,days_til_lockout,total_paid
589556,1351176,270,265,-13,52355
586196,1344957,160,268,-46,24160


In [46]:
%%sql
SELECT * FROM Person_democraphic LIMIT 2;

 * sqlite:///testdb.sqlite
Done.


customer_id,country,region,Profession
1351176,Benin,Oueme,Tehnician
1344957,Benin,Alibori,Doctor


In [16]:
%%sql
SELECT COUNT(Loan_id) FROM Loan_current_daitails;
SELECT COUNT(customer_id) FROM Loan_current_daitails;


 * sqlite:///testdb.sqlite
Done.
Done.


COUNT(customer_id)
5000


### Write a SQL code to determine the total number of loans by type of kit

In [28]:
%%sql
SELECT COUNT(*)
FROM(SELECT DISTINCT(product_type) FROM Sales_details);

 * sqlite:///testdb.sqlite
Done.


COUNT(*)
18


### Write SQL code to determine the number of inactive loans

In [29]:
%%sql
SELECT COUNT(*)
FROM(SELECT * FROM Loan_current_daitails
     WHERE days_til_lockout < 0);

 * sqlite:///testdb.sqlite
Done.


COUNT(*)
1294


### Write a SQL code to determine the regions where the total amount of payments exceeds 1,000,000

In [35]:
%%sql
SELECT paym.region, SUM(paym.total) total
FROM (SELECT pd.region region, SUM(ld.total_paid) total
      FROM Person_democraphic pd, Loan_current_daitails ld
      WHERE ld.customer_id = pd.customer_id
      GROUP BY region) paym
WHERE total > 1000000
GROUP BY paym.region


 * sqlite:///testdb.sqlite
Done.


region,total
Alibori,8559985
Atacora,7802685
Atlantique,51293780
Borgou,18947292
Collines,11904427
Couffo,4812070
Donga,9936982
Littoral,3980519
Mono,11621621
Oueme,30027194


### Write a SQL code to classify by region and in ascending order, the loans according to the total amount paid

In [36]:
%%sql
SELECT paym.region, SUM(paym.total) total
FROM (SELECT pd.region region, SUM(ld.total_paid) total
      FROM Person_democraphic pd, Loan_current_daitails ld
      WHERE ld.customer_id = pd.customer_id
      GROUP BY region) paym
GROUP BY paym.region
ORDER BY total ASC


 * sqlite:///testdb.sqlite
Done.


region,total
Littoral,3980519
Couffo,4812070
Atacora,7802685
Alibori,8559985
Zou,9540211
Plateau,9691206
Donga,9936982
Mono,11621621
Collines,11904427
Borgou,18947292



### Write a SQL code to determine the top 10 occupations by region with the highest total amount of payments

In [48]:
%%sql
SELECT Profession, COUNT(Profession ) AS 'value_occ'
FROM Person_democraphic
WHERE region IN ('Atlantique', 'Oueme', 'Borgou', 'Collines', 'Mono','Donga','Plateau','Zou', 'Alibori', 'Atacora')
GROUP BY Profession
ORDER BY value_occ DESC
LIMIT 10;

 * sqlite:///testdb.sqlite
Done.


Profession,value_occ
Nurse,502
Teacher,489
Statistian,480
Doctor,477
Ingenieur,476
Tehnician,475
Farmer,470
Security agent,459
Cmmercial,451
Driver,450
